## Install MMDetection

<https://mmdetection.readthedocs.io/en/latest/get_started.html>

## Download COCO VAL

In [1]:
!wget -P COCO/DIR/ http://images.cocodataset.org/annotations/annotations_trainval2017.zip

!wget -P COCO/DIR/ http://images.cocodataset.org/zips/val2017.zip

--2024-04-13 21:58:23--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.28.198, 3.5.25.88, 52.217.14.124, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.28.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘COCO/DIR/annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  14.9MB/s    in 26s     

2024-04-13 21:58:50 (9.27 MB/s) - ‘COCO/DIR/annotations_trainval2017.zip’ saved [252907541/252907541]

--2024-04-13 21:58:50--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.29.24, 52.216.52.249, 3.5.29.27, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.29.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘COCO/DIR/val2017.z

## Unzip COCO VAL

In [2]:
!unzip -qq COCO/DIR/annotations_trainval2017.zip -d COCO/DIR/

In [3]:
!unzip -qq COCO/DIR/val2017.zip -d COCO/DIR/

## Download model

In [4]:
import mmdet
import mmengine
import os
import os.path as osp

config_dir = osp.dirname(mmdet.__file__)
sub_config = "configs/rtmdet/rtmdet-ins_tiny_8xb32-300e_coco.py"
config_file = osp.join(config_dir, ".mim", sub_config)
cfg = mmengine.Config.fromfile(config_file)

model_file = "https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet-ins_tiny_8xb32-300e_coco/rtmdet-ins_tiny_8xb32-300e_coco_20221130_151727-ec670f7e.pth"

print(f"{config_file=}")
print(f"{model_file=}")

!mkdir -p -m 777 model

cfg.dump(osp.join("model", osp.basename(config_file)))
!wget -P model/ {model_file}

!ls -lah model

config_file='/home/mixaill76/.local/lib/python3.10/site-packages/mmdet/.mim/configs/rtmdet/rtmdet-ins_tiny_8xb32-300e_coco.py'
model_file='https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet-ins_tiny_8xb32-300e_coco/rtmdet-ins_tiny_8xb32-300e_coco_20221130_151727-ec670f7e.pth'
--2024-04-13 22:13:51--  https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet-ins_tiny_8xb32-300e_coco/rtmdet-ins_tiny_8xb32-300e_coco_20221130_151727-ec670f7e.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 104.166.182.22, 104.166.182.24, 104.166.182.27, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|104.166.182.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22757492 (22M) [application/octet-stream]
Saving to: ‘model/rtmdet-ins_tiny_8xb32-300e_coco_20221130_151727-ec670f7e.pth’

rtmdet-ins_tiny_8xb 100%[===================>]  21.70M  6.14MB/s    in 4.2s    

2024-04-13 22:13:57 (5.21 MB/s) - ‘model/rtmdet-ins_tiny_8xb32-300e_coc

## Validate

In [1]:
from mmdet.apis import inference_detector, init_detector
from mmengine.registry import init_default_scope
from mmdet.datasets import CocoDataset
import tqdm
import copy
import pickle
from coco_metric import CocoMetric

In [2]:
init_default_scope("mmdet")

In [3]:
model = init_detector(
    "./model/rtmdet-ins_tiny_8xb32-300e_coco.py",
    "./model/rtmdet-ins_tiny_8xb32-300e_coco_20221130_151727-ec670f7e.pth",
    device="cuda",
)

Loads checkpoint by local backend from path: ./model/rtmdet-ins_tiny_8xb32-300e_coco_20221130_151727-ec670f7e.pth


In [4]:
pipeline = [
    dict(type="LoadImageFromFile"),
    dict(type="mmdet.LoadAnnotations", with_bbox=True),
    dict(
        type="mmdet.PackDetInputs",
        meta_keys=(
            "img",
            "img_id",
            "img_path",
            "ori_shape",
            "img_shape",
            "instances",
        ),
    ),
]

dataset = CocoDataset(
    data_root="./COCO/DIR/",
    ann_file="annotations/instances_val2017.json",
    data_prefix=dict(img="val2017/"),
    pipeline=pipeline,
)
len(dataset)

loading annotations into memory...
Done (t=0.45s)
creating index...
index created!


5000

In [5]:
data_samples = []

for i in tqdm.tqdm(range(500)):
    item = dataset[i]
    data_sample = item["data_samples"]

    img = data_sample.img
    result = inference_detector(model, img)

    for key in result.pred_instances.all_keys():
        result.pred_instances[key] = result.pred_instances[key].detach().cpu()

    for bad_key in set(result.pred_instances.all_keys()) - {
        "masks",
        "scores",
        "labels",
        "bboxes",
    }:
        del result.pred_instances[bad_key]

    data_sample.pred_instances = result.pred_instances

    data_samples.append(data_sample.to_dict())

  0%|          | 0/500 [00:00<?, ?it/s]/home/mixaill76/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
/home/mixaill76/.local/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 500/500 [00:17<00:00, 29.16it/s]


## Generate tmp results

In [6]:
metric = CocoMetric(metric="bbox")
metric.dataset_meta = model.dataset_meta
metric.process({}, data_samples)
tmp_results = copy.deepcopy(metric.results)

In [7]:
with open("COCO/tmp_results.pkl", "wb") as fd:
    pickle.dump(tmp_results, fd)

with open("COCO/dataset_meta.pkl", "wb") as fd:
    pickle.dump(model.dataset_meta, fd)

## Test basic bbox pycocotools

In [8]:
!python3 coco_metric.py 'COCO/tmp_results.pkl' 'COCO/dataset_meta.pkl' --metric "bbox"

Namespace(tmp_results='COCO/tmp_results.pkl', dataset_meta='COCO/dataset_meta.pkl', metric='bbox', use_faster=False)
04/13 23:25:20 - mmengine - INFO - Converting ground truth to coco format...
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
04/13 23:25:21 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.72s).
Accumulating evaluation results...
DONE (t=0.61s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.452
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.631
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.492
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.452
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= lar

## Test bbox with Faster Coco Eval

In [9]:
!python3 coco_metric.py 'COCO/tmp_results.pkl' 'COCO/dataset_meta.pkl' --metric "bbox" --use_faster

Namespace(tmp_results='COCO/tmp_results.pkl', dataset_meta='COCO/dataset_meta.pkl', metric='bbox', use_faster=True)
04/13 23:25:29 - mmengine - INFO - Converting ground truth to coco format...
04/13 23:25:30 - mmengine - INFO - Evaluating bbox...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished...
DONE (t=0.59s).
Accumulating evaluation results...
COCOeval_opt.accumulate() finished...
DONE (t=0.15s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.452
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.631
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.492
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.452
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.594
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.609
 Average Recal

## Test basic segm pycocotools

In [10]:
!python3 coco_metric.py 'COCO/tmp_results.pkl' 'COCO/dataset_meta.pkl' --metric "segm"

Namespace(tmp_results='COCO/tmp_results.pkl', dataset_meta='COCO/dataset_meta.pkl', metric='segm', use_faster=False)
04/13 23:25:39 - mmengine - INFO - Converting ground truth to coco format...
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
04/13 23:25:40 - mmengine - INFO - Evaluating segm...
Loading and preparing results...
DONE (t=0.18s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=2.07s).
Accumulating evaluation results...
DONE (t=0.61s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.601
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.423
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.133
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.377
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= lar

## Test segm with Faster Coco Eval

In [11]:
!python3 coco_metric.py 'COCO/tmp_results.pkl' 'COCO/dataset_meta.pkl' --metric "segm" --use_faster

Namespace(tmp_results='COCO/tmp_results.pkl', dataset_meta='COCO/dataset_meta.pkl', metric='segm', use_faster=True)
04/13 23:25:48 - mmengine - INFO - Converting ground truth to coco format...
04/13 23:25:48 - mmengine - INFO - Evaluating segm...
Evaluate annotation type *segm*
COCOeval_opt.evaluate() finished...
DONE (t=0.76s).
Accumulating evaluation results...
COCOeval_opt.accumulate() finished...
DONE (t=0.15s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.601
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.423
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.133
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.377
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.553
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.524
 Average Recal

## Summary for 500 imgs

| Type | COCOeval    | COCOeval_faster | Profit      |
| ---- | ----------- | --------------- | ----------- |
| bbox | 3.57s | 2.03s      | ~1.7x faster |
| segm | 4.16s | 2.41s     | ~1.7x faster   |
